In [1]:
####1
# импорт библиотек
import os
import cv2
import easyocr
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import re
import pytesseract
from scipy.signal import wiener
from skimage.restoration import richardson_lucy
print("Импортирование завершено")

# пути к данным
DATA_PATH = '/kaggle/input/price-img-dataset'

IMG_FOLDER = os.path.join(DATA_PATH, 'imgs-20250131T163534Z-001', 'imgs')
TEST_CSV = os.path.join(DATA_PATH, 'test.csv')

# этот файл был создан путём ручной проверки какждого изображения, все значения данного файла истинные - 
# по нему и определим работоспособность модел
TRUE_TEST_CSV = os.path.join(DATA_PATH, 'true_test_predictions.csv')  # содержит столбцы: img_name и true_price


# Для проверки работы на валидации
#VAL_CSV = os.path.join(DATA_PATH, 'val.csv')
#TRUE_VAL_CSV = os.path.join(DATA_PATH, 'true_val_predictions.csv')

# Для проверки работы на трейне
#TRAIN_CSV = os.path.join(DATA_PATH, 'train.csv')
#TRUE_VAL_CSV = os.path.join(DATA_PATH, 'train_val_predictions.csv')


Импортирование завершено


### Почему выбраны EasyOCR и Tesseract вместо моделей на базе ResNet

Протестировал различные подходы для распознавания цены на изображениях. Подход с регрессионными моделями (например, на базе ResNet) показал низкую точность, поскольку извлечение числовых значений напрямую из изображений оказалось сложной задачей. Поэтому перешёл к использованию готовых OCR-движков – EasyOCR и Tesseract, которые специализированы на распознавании текста и, особенно, цифр, что позволяет добиться значительно лучших результатов.

In [2]:
####2
# повышаем контрастность (CLAHE) с морфологической обработкой
def enhance_contrast(image):
    # приводим изображение к grayscale
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    # морфологическая обработка
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    closed = cv2.morphologyEx(enhanced, cv2.MORPH_CLOSE, kernel)
    return closed

# доп предобработка -  (1) увеличение, (2) медианный фильтр и (3) адаптивная бинаризация
def preprocess_for_ocr(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # (1) увеличение
    resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    # (2) медианный фильтр 
    blurred = cv2.medianBlur(resized, 3)
    # (3) адаптивная бинаризация
    adaptive = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
    return adaptive

In [3]:
# Функция Unsharp Masking
def unsharp_mask(image, kernel_size=(5,5), sigma=1.0, amount=1.0, threshold=0):
    # Применяем гауссово размытие
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    # Вычисляем маску резкости
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

# Функция для коррекции гаммы (gamma correction)
def adjust_gamma(image, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Функция применения билинейного фильтра для сглаживания с сохранением краев
def apply_bilateral_filter(image):
    return cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)


In [4]:
####3
# функция для поворота изображения на заданный угол
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# функция очистки от типичных ошибок OCR
def clean_candidate(candidate):
    #candidate = candidate.replace('O', '0').replace('o', '0')
    #candidate = candidate.replace('l', '1').replace('I', '1')
    #candidate = candidate.strip()
    return candidate
    
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

In [5]:
###4
# получаем кандидатов с помощью EasyOCR
def get_candidates_easyocr(image, confidence_threshold):
    candidates = []
    results = reader.readtext(image)
    texts = [res[1] for res in results if res[2] > confidence_threshold]
    combined_text = "".join(texts)
    digits_found = re.findall(r'\d+', combined_text)
    for digits in digits_found:
        cleaned = clean_candidate(digits)
        candidates.append(cleaned)
    return candidates

# получаем кандидатов с помощью Tesseract OCR
def get_candidates_tesseract(image):

    if len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
        
    # gрименяем Tesseract
    config = '--psm 6'  
    text = pytesseract.image_to_string(gray, config=config)

    candidates = re.findall(r'\d+', text)

    candidates = [clean_candidate(cand) for cand in candidates]
    return candidates

In [6]:
def correct_perspective(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    if not contours:
        return image  # Если контуры не найдены, возвращаем исходное изображение

    approx = cv2.approxPolyDP(contours[0], 0.02 * cv2.arcLength(contours[0], True), True)

    if len(approx) == 4:  # Если найдено 4 точки (прямоугольник)
        pts_src = np.float32([point[0] for point in approx])
        pts_dst = np.float32([[0, 0], [image.shape[1], 0], [image.shape[1], image.shape[0]], [0, image.shape[0]]])
        matrix = cv2.getPerspectiveTransform(pts_src, pts_dst)
        corrected = cv2.warpPerspective(image, matrix, (image.shape[1], image.shape[0]))
        return corrected
    return image  # Если не удалось скорректировать, возвращаем оригинал

def wiener_deblur(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    deblurred = wiener(gray, (5, 5))  # 5x5 — размер ядра для Винера
    return cv2.cvtColor(np.uint8(deblurred), cv2.COLOR_GRAY2BGR)

def richardson_lucy_deblur(image, iterations=10):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    psf = np.ones((5, 5)) / 25  # Оценка функции рассеяния точки (PSF)
    deblurred = richardson_lucy(gray, psf, iterations)
    return cv2.cvtColor(np.uint8(deblurred * 255), cv2.COLOR_GRAY2BGR)


### Ансамблирование результатов OCR

Для повышения точности использую ансамбль двух OCR-систем – EasyOCR и Tesseract. Для каждого варианта предобработки изображения (например, CLAHE с морфологической обработкой, дополнительные методы предобработки, повороты) извлекаются кандидаты на числовое значение. Затем объединяем их и выбираем тот, который встречается с наибольшей "весовой" оценкой (суммарная уверенность), что позволяет компенсировать систематические ошибки отдельных моделей.

In [7]:
###5
# predict функция
def predict_price_ocr(img_path, confidence_threshold=0.4):
    """
    Функция пытается распознать цену с изображения, используя ансамбль из EasyOCR и Tesseract.
    Варианты предобработки:
      1. Оригинальное изображение.
      2. CLAHE + морфология.
      3. Дополнительная предобработка (увеличение, медианный фильтр, адаптивная бинаризация).
      4. Повороты CLAHE-обработанного изображения на ±5°, ±10°, ±15°, ±35°.
      5. Unsharp Masking.
      6. Gamma Correction.
      7. Bilateral Filter.
      8. Коррекция перспективы.
      9. Восстановление размытия (фильтр Винера).
      10. Восстановление размытия (Richardson-Lucy).
    Полученные кандидаты объединяются, и выбирается тот, который встречается с наибольшей частотой
    (при равенстве – выбирается самый длинный). Если кандидаты не получены, возвращается None.
    """
    image = cv2.imread(img_path)
    if image is None:
        raise ValueError(f"Не удалось загрузить изображение: {img_path}")
    
    variants = []
    # (1) оригинальное изображение
    variants.append(image)

    # (2) CLAHE + морфология
    variant1 = enhance_contrast(image)
    variants.append(variant1)

    # (3) дополнительная предобработка
    variants.append(preprocess_for_ocr(image))
    
    # (4) с поворотами CLAHE
    variants.append(rotate_image(variant1, 5))
    variants.append(rotate_image(variant1, -5))
    variants.append(rotate_image(variant1, 10))
    variants.append(rotate_image(variant1, -10))
    variants.append(rotate_image(variant1, 15))
    variants.append(rotate_image(variant1, -15))
    # (5) вариант с Unsharp Masking
    def unsharp_mask(image, kernel_size=(5,5), sigma=1.0, amount=1.0, threshold=0):
        blurred = cv2.GaussianBlur(image, kernel_size, sigma)
        sharpened = float(amount + 1) * image - float(amount) * blurred
        sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
        sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
        sharpened = sharpened.round().astype(np.uint8)
        if threshold > 0:
            low_contrast_mask = np.absolute(image - blurred) < threshold
            np.copyto(sharpened, image, where=low_contrast_mask)
        return sharpened
    unsharp = unsharp_mask(image)
    variants.append(unsharp)
    # (6) вариант с Gamma Correction
    def adjust_gamma(image, gamma=1.5):
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)
    gamma_corrected = adjust_gamma(image, gamma=1.5)
    variants.append(gamma_corrected)
    # (7) вариант с Bilateral Filter (сглаживание с сохранением краев)
    bilateral = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    variants.append(bilateral)
    # (8) Коррекция перспективы
    perspective_corrected = correct_perspective(image)
    #variants.append(perspective_corrected)

    # (9) Восстановление размытия (фильтр Винера)
    wiener_restored = wiener_deblur(image)
    variants.append(wiener_restored)

    # (10) Восстановление размытия (метод Richardson-Lucy)
    richardson_lucy_restored = richardson_lucy_deblur(image)
    variants.append(richardson_lucy_restored)

    all_candidates = []
    
    for variant in variants:
        # Получаем кандидатов от EasyOCR
        candidates_e = get_candidates_easyocr(variant, confidence_threshold)
        all_candidates.extend(candidates_e)
        # Получаем кандидатов от Tesseract
        candidates_t = get_candidates_tesseract(variant)
        all_candidates.extend(candidates_t)
    
    if not all_candidates:
        return None
    
    # Подсчитываем частоту каждого кандидата
    freq = {}
    for cand in all_candidates:
        # Можно ограничить длину кандидата, если известно, что цена состоит из 2-4 цифр
        if len(cand) < 2 or len(cand) > 4:
            continue
        freq[cand] = freq.get(cand, 0) + 1

    if not freq:
        return None
    
    best_candidate = None
    best_freq = 0
    for cand, count in freq.items():
        if count > best_freq or (count == best_freq and (best_candidate is None or len(cand) > len(best_candidate))):
            best_candidate = cand
            best_freq = count

    if best_candidate is None or best_candidate == "":
        return None
    return int(best_candidate)


In [8]:
####5.5 предсказания на валидационном (трейн) наборе (занимает ~ 40 минут)
#val_df = pd.read_csv(TRAIN_CSV)   #train_df = pd.read_csv(TRAIN_CSV) - заменить при проверки на трейн датасете (далее тоже самое)
#predicted_prices = []

#for img_name in tqdm(val_df['img_name'], desc="Predicting VAL set with OCR"):
#    img_path = os.path.join(IMG_FOLDER, img_name)
#    price = predict_price_ocr(img_path, confidence_threshold=0.35)
#    if price is None:
#        price = '0'
#    predicted_prices.append(price)

#pred_df = val_df[['img_name']].copy() # pred_df = train_df[['img_name']].copy()
#pred_df['predicted_price'] = predicted_prices
#pred_df.to_csv("val_predictions_ocr.csv", index=False) # pred_df.to_csv("train_predictions_ocr.csv", index=False) 

#### Сравнение с истинными значениями VAL/TRAIN набора

#merged = val_df.merge(pred_df, on='img_name', how='left') # merged = train_df.merge(pred_df, on='img_name', how='left')
#merged.to_csv("val_predictions_with_true_ocr.csv", index=False) # merged.to_csv("train_predictions_with_true_ocr.csv", index=False)

#merged['correct'] = merged.apply(lambda row: (row['predicted_price'] != '0') and (row['text'] == row['text']), axis=1)
#num_correct = merged['correct'].sum()
#num_total = len(merged)
#num_incorrect = num_total - num_correct
#print(f"Верных ответов: {num_correct}, Неверных ответов: {num_incorrect}")
#Лучший результат: "Верных ответов: 932, Неверных ответов : 68" на  - на валидации
#Лучший результат: "Верных ответов: 4916, Неверных ответов : 36" на  - на трейне

In [9]:
###6
# предсказание на тестовом наборе с использованием OCR (занимает ~ 20 минут на CPU)
test_df = pd.read_csv(TEST_CSV)
predicted_prices = []
unknown_images = []  # список изображений, для которых не удалось распознать число

for img_name in tqdm(test_df['img_name'], desc="Predicting test set with OCR"):
    img_path = os.path.join(IMG_FOLDER, img_name)
    price = predict_price_ocr(img_path, confidence_threshold=0.35)
    if price is None:
        price = '0'
        unknown_images.append(img_name)
    predicted_prices.append(f"{float(price):.1f}")

test_df['predicted_price'] = predicted_prices
test_df['predicted_price'] = test_df['predicted_price'].astype(float)
test_df.to_csv("submission.csv", index=False)


Predicting test set with OCR:  95%|█████████▌| 227/238 [19:56<00:54,  4.91s/it]/usr/local/lib/python3.10/dist-packages/scipy/signal/_signaltools.py:1659: RuntimeWarning: divide by zero encountered in divide
  res *= (1 - noise / lVar)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_signaltools.py:1659: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)
<ipython-input-6-b3ea9716a713>:24: RuntimeWarning: invalid value encountered in cast
  return cv2.cvtColor(np.uint8(deblurred), cv2.COLOR_GRAY2BGR)
Predicting test set with OCR: 100%|██████████| 238/238 [20:59<00:00,  5.29s/it]


In [10]:
###7
# сравнение с истинными значениями
true_test_df = pd.read_csv(TRUE_TEST_CSV)  
merged = true_test_df.merge(test_df, on='img_name', how='left')
merged['true_price'] = merged['true_price'].astype(float)
merged['predicted_price'] = merged['predicted_price'].astype(float)
merged.to_csv("test_predictions_with_true_ocr.csv", index=False)

# ответ считается верным, если predicted_price точно совпадает с true_price
merged['correct'] = merged.apply(lambda row: (row['predicted_price'] != '0') and (row['true_price'] == row['predicted_price']), axis=1)
num_correct = merged['correct'].sum()
num_total = len(merged)
num_incorrect = num_total - num_correct

# Формируем список изображений, для которых модель не смогла предсказать цену корректно или предсказала 0
error_images = merged[(merged['predicted_price'] == "0") | (~merged['correct'])]['img_name'].tolist()

print(f"Верных ответов: {num_correct}, Неверных ответов: {num_incorrect}")
#Лучший результат: "Верных ответов: 230, Неверных ответов: 8" на CPU

print("Изображения, для которых модель не смогла предсказать цену:")
for img in error_images:
    print(img)

Верных ответов: 230, Неверных ответов: 8
Изображения, для которых модель не смогла предсказать цену:
510883758_2.jpg
510880074_2.jpg
481897095_2.jpg
483319459_2.jpg
508780913_2.jpg
508757462_2.jpg
510885952_2.jpg
481896944_2.jpg
